<!-- ---
title: Hyperparameter Tuning in Ignite
date: 2021-09-28
downloads: true
sidebar: true
tags:
  - hyperparameter tuning
  - ray tune
  - optuna
  - ax
--- -->

#  Hyperparameter Tuning in Ignite

In this tutorial, we will see how [Ray Tune](https://docs.ray.io/en/stable/tune.html) can be used with Ignite for hyperparameter tuning. We will also compare it with other frameworks like [Optuna](https://optuna.org/) and [Ax](https://ax.dev/) for hyperparameter optimization.

<!--more-->

We will follow [this PyTorch tutorial](https://pytorch.org/tutorials/beginner/hyperparameter_tuning_tutorial.html) for reference.

In this example, we will be using a [ResNet18](https://pytorch.org/vision/stable/models.html#torchvision.models.resnet18) model on the [MNIST](https://pytorch.org/vision/stable/datasets.html#torchvision.datasets.MNIST) dataset. The base code is the same as used in the [Getting Started Guide](https://pytorch-ignite.ai/tutorials/getting-started/).

## Required Dependencies

In [ ]:
!pip install pytorch-ignite
!pip install ray

## Imports

In [2]:
import os
from functools import partial
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import random_split

from torch.utils.data import DataLoader, SubsetRandomSampler
from torchvision.datasets import CIFAR10
from torchvision.models import resnet18
from torchvision.transforms import Compose, Normalize, ToTensor

from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss, RunningAverage
import ignite.distributed as idist

from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler

In [3]:
def load_data(data_dir="./data"):
    transform = Compose([
        ToTensor(),
        Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    trainset = CIFAR10(
        root=data_dir, train=True, download=True, transform=transform)

    testset = CIFAR10(
        root=data_dir, train=False, download=True, transform=transform)

    return trainset, testset

In [4]:
class Net(nn.Module):
    def __init__(self, l1=120, l2=84):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, l1)
        self.fc2 = nn.Linear(l1, l2)
        self.fc3 = nn.Linear(l2, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [13]:
def train_cifar(config, checkpoint_dir=None, data_dir=None):
    net = idist.auto_model(Net(config["l1"], config["l2"]))

    device = idist.device()

    criterion = nn.CrossEntropyLoss()
    optimizer = idist.auto_optim(optim.SGD(net.parameters(), lr=config["lr"], momentum=0.9))

    trainer = create_supervised_trainer(net, optimizer, criterion, device=device, non_blocking=True)
    val_evaluator = create_supervised_evaluator(net, metrics={ "accuracy": Accuracy(), "loss": Loss(criterion)}, device=device, non_blocking=True)

    to_save = { "model": net, "optimizer": optimizer}

    if checkpoint_dir:
        model_state, optimizer_state = torch.load(
            os.path.join(checkpoint_dir, "checkpoint"))
        net.load_state_dict(model_state)
        optimizer.load_state_dict(optimizer_state)

    trainset, testset = load_data(data_dir)

    test_abs = int(len(trainset) * 0.8)
    train_subset, val_subset = random_split(
        trainset, [test_abs, len(trainset) - test_abs])

    trainloader = idist.auto_dataloader(
        train_subset,
        batch_size=int(config["batch_size"]),
        shuffle=True,
        num_workers=8)
    valloader = idist.auto_dataloader(
        val_subset,
        batch_size=int(config["batch_size"]),
        shuffle=True,
        num_workers=8)

    avg_output = RunningAverage(output_transform=lambda x: x)
    avg_output.attach(trainer, 'running_avg_loss')

    # handler = Checkpoint(
    #     to_save, DiskSaver('models', create_dir=True), n_saved=2, global_step_transform=gst
    # )
    # trainer.add_event_handler(Events.EPOCH_COMPLETED, handler)

    @trainer.on(Events.ITERATION_COMPLETED(every=2000))
    def log_training_loss(engine):
        print(f"Epoch[{engine.state.epoch}], Iter[{engine.state.iteration}] Loss: {engine.state.output:.2f} Running Avg Loss: {engine.state.metrics['running_avg_loss']:.2f}")


    @trainer.on(Events.EPOCH_COMPLETED)
    def log_validation_results(trainer):
        val_evaluator.run(valloader)
        metrics = val_evaluator.state.metrics
        print(f"Validation Results - Epoch[{trainer.state.epoch}] Avg accuracy: {metrics['accuracy']:.2f} Avg loss: {metrics['loss']:.2f}")

        with tune.checkpoint_dir(trainer.state.epoch) as checkpoint_dir:
          path = os.path.join(checkpoint_dir, "checkpoint")
          torch.save((net.state_dict(), optimizer.state_dict()), path)
        tune.report(loss=metrics['loss'], accuracy=metrics['accuracy'])   

    trainer.run(trainloader, max_epochs=10) 

        
    print("Finished Training")

In [14]:
def main(num_samples=10, max_num_epochs=10, gpus_per_trial=1):
    data_dir = os.path.abspath("./data")
    trainset, testset = load_data(data_dir)
    config = {
        "l1": tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),
        "l2": tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),
        "lr": tune.loguniform(1e-4, 1e-1),
        "batch_size": tune.choice([2, 4, 8, 16])
    }
    scheduler = ASHAScheduler(
        metric="loss",
        mode="min",
        max_t=max_num_epochs,
        grace_period=1,
        reduction_factor=2)
    reporter = CLIReporter(
        # parameter_columns=["l1", "l2", "lr", "batch_size"],
        metric_columns=["loss", "accuracy", "training_iteration"])
    result = tune.run(
        partial(train_cifar, data_dir=data_dir),
        resources_per_trial={"cpu": 2, "gpu": gpus_per_trial},
        config=config,
        num_samples=num_samples,
        scheduler=scheduler,
        progress_reporter=reporter)

    best_trial = result.get_best_trial("loss", "min", "last")
    print("Best trial config: {}".format(best_trial.config))
    print("Best trial final validation loss: {}".format(
        best_trial.last_result["loss"]))
    print("Best trial final validation accuracy: {}".format(
        best_trial.last_result["accuracy"]))

    best_trained_model = idist.auto_model(Net(best_trial.config["l1"], best_trial.config["l2"]))
    device = idist.device()

    best_checkpoint_dir = best_trial.checkpoint.value
    model_state, optimizer_state = torch.load(os.path.join(
        best_checkpoint_dir, "checkpoint"))
    best_trained_model.load_state_dict(model_state)

    test_evaluator = create_supervised_evaluator(best_trained_model, metrics={"Accuracy": Accuracy()}, device=device, non_blocking=True)

    trainset, testset = load_data()

    testloader = idist.auto_dataloader(testset, batch_size=4, shuffle=False, num_workers=2)

    test_evaluator.run(testloader)
    print(test_evaluator.state.metrics)

In [15]:
main(num_samples=3, max_num_epochs=3, gpus_per_trial=1)

Files already downloaded and verified
Files already downloaded and verified


2021-10-01 11:50:00,377	WARNING experiment.py:296 -- No name detected on trainable. Using DEFAULT.
2021-10-01 11:50:00,379	INFO registry.py:67 -- Detected unknown callable for trainable. Converting to class.
2021-10-01 11:50:00,396	WARNING callback.py:117 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`


== Status ==
Memory usage on this node: 2.2/12.7 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 2.000: None | Iter 1.000: None
Resources requested: 0/2 CPUs, 0/1 GPUs, 0.0/7.33 GiB heap, 0.0/3.67 GiB objects (0.0/1.0 accelerator_type:K80)
Result logdir: /root/ray_results/DEFAULT_2021-10-01_11-50-00
Number of trials: 3/3 (3 PENDING)
+---------------------+----------+-------+--------------+------+------+------------+
| Trial name          | status   | loc   |   batch_size |   l1 |   l2 |         lr |
|---------------------+----------+-------+--------------+------+------+------------|
| DEFAULT_b50ea_00000 | PENDING  |       |            8 |   16 |    4 | 0.00059636 |
| DEFAULT_b50ea_00001 | PENDING  |       |            2 |   32 |   64 | 0.00074678 |
| DEFAULT_b50ea_00002 | PENDING  |       |            2 |   64 |   32 | 0.00122324 |
+---------------------+----------+-------+--------------+------+------+------------+


(pid=1853) Files already downloaded and verified
== Status ==


(pid=1853) 2021-10-01 11:50:06,682 ignite.distributed.auto.auto_dataloader INFO: Use data loader kwargs for dataset '<torch.utils.data.da': 
(pid=1853) 	{'batch_size': 8, 'shuffle': True, 'num_workers': 8, 'pin_memory': True}
(pid=1853) /usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
(pid=1853)   cpuset_checked))
(pid=1853) 2021-10-01 11:50:06,683 ignite.distributed.auto.auto_dataloader INFO: Use data loader kwargs for dataset '<torch.utils.data.da': 
(pid=1853) 	{'batch_size': 8, 'shuffle': True, 'num_workers': 8, 'pin_memory': True}
(pid=1853) /usr/local/lib/python3.7/dist-packages/torch/nn/functional.

(pid=1853) Epoch[1], Iter[2000] Loss: 2.31 Running Avg Loss: 2.31
(pid=1853) Epoch[1], Iter[4000] Loss: 2.31 Running Avg Loss: 2.30


...

(pid=2361) Epoch[3], Iter[42000] Loss: 2.52 Running Avg Loss: 1.34
(pid=2361) Epoch[3], Iter[44000] Loss: 1.35 Running Avg Loss: 1.32
(pid=2361) Epoch[3], Iter[46000] Loss: 1.74 Running Avg Loss: 1.36
(pid=2361) Epoch[3], Iter[48000] Loss: 1.13 Running Avg Loss: 1.45
(pid=2361) Epoch[3], Iter[50000] Loss: 1.30 Running Avg Loss: 1.40
(pid=2361) Epoch[3], Iter[52000] Loss: 1.87 Running Avg Loss: 1.24
(pid=2361) Epoch[3], Iter[54000] Loss: 0.37 Running Avg Loss: 1.19
(pid=2361) Epoch[3], Iter[56000] Loss: 0.92 Running Avg Loss: 1.37
(pid=2361) Epoch[3], Iter[58000] Loss: 0.80 Running Avg Loss: 1.36
(pid=2361) Epoch[3], Iter[60000] Loss: 1.18 Running Avg Loss: 1.38


(pid=2361) [W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
(pid=2361) [W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
(pid=2361) [W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
(pid=2361) [W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
(pid=2361) [W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
(pid=2361) [W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
(pid=2361) [W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
(pid=2361) [W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
2021-10-01 12:05:05,896	INFO tune.py:561 -- Total run time: 905.52 seconds (905.37 seconds for the tuning loop).


Result for DEFAULT_b50ea_00002:
  accuracy: 0.5291
  date: 2021-10-01_12-05-05
  done: true
  experiment_id: 1c5539c7b0e24aef998eaca10497f948
  hostname: ac982acc27c2
  iterations_since_restore: 3
  loss: 1.33420810546875
  node_ip: 172.28.0.2
  pid: 2361
  should_checkpoint: true
  time_since_restore: 389.19638776779175
  time_this_iter_s: 128.4807460308075
  time_total_s: 389.19638776779175
  timestamp: 1633089905
  timesteps_since_restore: 0
  training_iteration: 3
  trial_id: b50ea_00002
  
== Status ==
Memory usage on this node: 3.4/12.7 GiB
Using AsyncHyperBand: num_stopped=3
Bracket: Iter 2.000: -1.4066556640625 | Iter 1.000: -1.46265712890625
Resources requested: 2.0/2 CPUs, 1.0/1 GPUs, 0.0/7.33 GiB heap, 0.0/3.67 GiB objects (0.0/1.0 accelerator_type:K80)
Result logdir: /root/ray_results/DEFAULT_2021-10-01_11-50-00
Number of trials: 3/3 (1 RUNNING, 2 TERMINATED)
+---------------------+------------+-----------------+--------------+------+------+------------+---------+----------

2021-10-01 12:05:07,794 ignite.distributed.auto.auto_dataloader INFO: Use data loader kwargs for dataset 'Dataset CIFAR10': 
	{'batch_size': 4, 'shuffle': False, 'num_workers': 2, 'pin_memory': True}
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


{'Accuracy': 0.5379}
